# Yo look here !

### Problem 2  simulate SIR spread on the network

In [1]:
import networkx as nx
import ndlib.models.epidemics as ep
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
from ndlib.viz.bokeh.MultiPlot import MultiPlot
from bokeh.io import output_notebook, show
import ndlib.models.ModelConfig as mc

# Network Definition (random network models)
G1 = nx.erdos_renyi_graph(1000, 0.1)
G2 = nx.barabasi_albert_graph(1000, 5)   
# n=nodes, m=#edges to attach from a new node to existing nodes
G3 = nx.watts_strogatz_graph(1000,10,0.1)
# n=nodes, k= node connected to k nearest neighbors, p=Probability of rewiring each edge

all_trends = {}
all_results = {}
initial_status = {i:1 for i in range(10)}

# 1 vary fraction_infected
# 2 specifically infect certain nodes
# 3 vary disease parameters 

betas = [0.03, 0.04, 0.05]
gammas = [0.015, 0.016, 0.017]
fractions = [0.01, 0.05, 0.1, 0.2]

for fraction in fractions:
    for beta in betas:
        for gamma in gammas:
            config = mc.Configuration()
            config.add_model_initial_configuration("Infected", initial_status)
            config.add_model_parameter('beta', beta)
            config.add_model_parameter('gamma', gamma)
            config.add_model_parameter("fraction_infected", fraction)

            model1 = ep.SIRModel(G1)
            model2 = ep.SIRModel(G2)
            model3 = ep.SIRModel(G3)
            
            model1.set_initial_status(config)
            model2.set_initial_status(config)
            model3.set_initial_status(config)

            iterations1 = model1.iteration_bunch(200)
            iterations2 = model2.iteration_bunch(200)
            iterations3 = model3.iteration_bunch(200)

            trends1 = model1.build_trends(iterations1)
            trends2 = model2.build_trends(iterations2)
            trends3 = model3.build_trends(iterations3)

            peak_infected1 = max(trends1[0]['trends']['node_count'][1])
            peak_infected2 = max(trends2[0]['trends']['node_count'][1])
            peak_infected3 = max(trends3[0]['trends']['node_count'][1])

            # storing the peak of the epidemic, which is the highest number of infected nodes at any time step during the simulation. 
            all_results[(fraction, beta, gamma)] = {
                'trends1': trends1,
                'trends2': trends2,
                'trends3': trends3,
                'peak_infected1': peak_infected1,
                'peak_infected2': peak_infected2,
                'peak_infected3': peak_infected3
            }


times1 = range(len(trends1[0]['trends']['node_count'][1]))
infected1 = trends1[0]['trends']['node_count'][1]

times2 = range(len(trends2[0]['trends']['node_count'][1]))
infected2 = trends2[0]['trends']['node_count'][1]

times3 = range(len(trends3[0]['trends']['node_count'][1]))
infected3 = trends3[0]['trends']['node_count'][1]


# Create a 3x1 subplot layout
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(16, 18))

# Plot the diffusion trend for model1
axes[0].plot(times1, infected1, '-o', label='Infected')
axes[0].set_title('Erdos-Renyi Graph',fontsize=10, y=1.0)
axes[0].set_ylabel('Number of Infected Nodes',fontsize=10, y=1.0)
axes[0].legend()

# Plot the diffusion trend for model2
axes[1].plot(times2, infected2, '-o', label='Infected')
axes[1].set_title('Barabasi-Albert Graph',fontsize=10, y=1.0)
axes[1].set_ylabel('Number of Infected Nodes',fontsize=10, y=1.0)
axes[1].legend()

# Plot the diffusion trend for model3
axes[2].plot(times3, infected3, '-o', label='Infected')
axes[2].set_title('Watts-Strogatz Graph',fontsize=10, y=1.0)
axes[2].set_ylabel('Number of Infected Nodes',fontsize=10, y=1.0)
axes[2].set_xlabel('Time')
axes[2].legend()

# Adjust the layout and display
plt.tight_layout()
plt.show()

no display found. Using non-interactive Agg backend


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19488\2364358282.py:104: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
